In [1]:
import sys
# !conda install --yes --prefix {sys.prefix} pyarrow

from collections import OrderedDict
from collections import Counter
import fastparquet
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import os
import openpyxl
import pandas as pd
import pyarrow as pa
# import pyarrow.parquet as pq
import random
import re
import seaborn as sns
import scipy.stats
from sklearn import preprocessing
import statistics


sns.set_style("whitegrid")

In [2]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

In [3]:
taskDF = pd.read_parquet(filePath)

In [4]:
filePath = os.path.join(tempTopDir, 'GP_Consent_Long.parquet')
consentDF_L = pd.read_parquet(filePath).infer_objects()
consentDF_L = consentDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID'].notna()]
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Consent_Short.parquet')
consentDF_S = pd.read_parquet(filePath).infer_objects()
consentDF_S = consentDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID'].notna()]
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Demographics_Long.parquet')
demoDF_L = pd.read_parquet(filePath).infer_objects()
demoDF_L = demoDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID'].notna()]
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_Demographics_Short.parquet')
demoDF_S = pd.read_parquet(filePath).infer_objects()
demoDF_S = demoDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID'].notna()]
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_AI_Long.parquet')
aiDF_L = pd.read_parquet(filePath).infer_objects()
aiDF_L = aiDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID'].notna()]
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID']!='nan']

filePath = os.path.join(tempTopDir, 'GP_AI_Short.parquet')
aiDF_S = pd.read_parquet(filePath).infer_objects()
aiDF_S = aiDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID'].notna()]
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID']!='nan']

In [5]:
print(len(demoDF_L.ParticipantPrivateID.unique()))
print(demoDF_L.ExperimentVersion.unique())

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [6]:
print(len(taskDF.ParticipantPrivateID.unique()))
print(taskDF.ExperimentVersion.unique())
print(aiDF_L.ExperimentVersion.unique())
print(consentDF_L.ExperimentVersion.unique())

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]


In [7]:
taskDF.ExperimentVersion.unique()

array([60., 62., 64., 65., 67., 70., 72., 73., 76., 77.])

In [8]:
# how are comp questions kept track of?
print(taskDF.Question.unique())
print(taskDF.columns.tolist())
print(taskDF[taskDF['Question']==20])
# print(taskDF[taskDF['CompConf']>0])
print(taskDF['Question'].unique())

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp', 'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion', 'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID', 'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus', 'ParticipantCompletionCode', 'ParticipantExternalSessionID', 'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS', 'ParticipantBrowser', 'ParticipantMonitorSize', 'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion', 'VisCond_DIFF', 'AgentCond_BlurVis_DIFF', 'AgentCond_Vis_DIFF', 'AgentCond_Blur_DIFF', 'checkpoint_ph4w', 'checkpoint_drrq', 'AgentCond_Norm_DIFF', 'checkpoint_6gsl', 'checkpoint_apgi', 'LVL', 'VisCond_EASY', 'AgentCond_Norm_EASY', 'AgentCond_Vis_EASY', 'AgentCond_BlurVis_EASY', 'AgentCond_Blur_EASY', 'Spreadsheet', 'SpreadsheetName', 'SpreadsheetRow', 'TrialNumber', 'ScreenNumber', 'ScreenName', 'ZoneNam

In [9]:
# # Make a second dataframe to hold "short" version of MainDF
# sm_data = OrderedDict()
# verbose = 0

# for p_index, p_num in enumerate(taskDF.ParticipantPrivateID.unique()):
#     if (p_num != 'nan'):
#         temp = taskDF[taskDF['ParticipantPrivateID']==p_num]

#         tmpConsent = consentDF_L[consentDF_L['ParticipantPrivateID']==p_num]
#         tmpConsent.set_index('EventIndex')
#         tmpDemo_L = demoDF_L[demoDF_L['ParticipantPrivateID']==p_num]
#         tmpDemo_L.set_index('EventIndex')
#         tmpAI_L = aiDF_L[aiDF_L['ParticipantPrivateID']==p_num]
#         tmpAI_L.set_index('EventIndex')

#         lvl = temp.LVL.unique()[0]
#         vc = temp.VisCond.unique()[0]
#         ag = temp.AgentCond.unique()[0]
#         temp1 = temp.query('Question == 1 & ZoneType == "response_button_text" & display == "Prediction" & ScreenName == "Screen 1"')
# #         print(temp1.columns)
# #         print(temp1.iloc[:,52:57])


tmp_DF = taskDF.query('ZoneType == "response_button_text" & Question >= 1')[['ParticipantPrivateID',\
                         'Question','Correct']]
print(tmp_DF.head(20))
tmp_DF.pivot(index='ParticipantPrivateID', columns='Question', values='Correct')






    ParticipantPrivateID  Question  Correct
39             3109877.0       1.0      1.0
40             3109877.0       2.0      0.0
41             3109877.0       3.0      0.0
42             3109877.0       4.0      1.0
47             3109877.0       5.0      0.0
48             3109877.0       6.0      0.0
55             3109877.0       7.0      0.0
56             3109877.0       8.0      0.0
71             3109877.0       9.0      0.0
72             3109877.0      10.0      0.0
73             3109877.0      11.0      0.0
74             3109877.0      12.0      0.0
75             3109877.0      13.0      0.0
76             3109877.0      14.0      0.0
84             3109877.0      15.0      1.0
85             3109877.0      16.0      1.0
101            3109877.0      17.0      0.0
107            3109877.0      18.0      1.0
122            3109877.0      19.0      0.0
141            3109877.0      20.0      0.0


Question,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0
ParticipantPrivateID,,,,,,,,,,,,,,,,,,,,
3109877.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3109878.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3109883.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3135747.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3160810.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3532055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3532499.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3532504.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [10]:
# Make a second dataframe to hold "short" version of MainDF
sm_data = OrderedDict()
verbose = 0

for p_index, p_num in enumerate(taskDF.ParticipantPrivateID.unique()):
    if (p_num != 'nan'):
        temp = taskDF[taskDF['ParticipantPrivateID']==p_num]

        tmpConsent = consentDF_L[consentDF_L['ParticipantPrivateID']==p_num]
        tmpConsent.set_index('EventIndex')
        tmpDemo_L = demoDF_L[demoDF_L['ParticipantPrivateID']==p_num]
        tmpDemo_L.set_index('EventIndex')
        tmpAI_L = aiDF_L[aiDF_L['ParticipantPrivateID']==p_num]
        tmpAI_L.set_index('EventIndex')

        lvl = temp.LVL.unique()[0]
        vc = temp.VisCond.unique()[0]
        ag = temp.AgentCond.unique()[0]

        temp1 = temp.query('ZoneType == "response_button_text" & display == "Prediction" & ScreenName == "Screen 1"')
        predTot = len(temp1)
        predCorr = temp1['Correct'].sum()
        if verbose:
            print("\ncorrect predictions:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(predCorr))

        temp1 = temp.query('AgentNum == 1 & ZoneType == "response_button_text" & display == "Prediction" & ScreenName == "Screen 1"')
        pred1Tot = len(temp1)
        pred1Corr = temp1['Correct'].sum()
        if verbose:
            print("correct predictions 1:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(pred1Corr))
#             if (pred1Corr > 0):
#                 print("Predictions1: ")
#                 print(pred1Corr)

        temp1 = temp.query('AgentNum == 2 & ZoneType == "response_button_text" & display == "Prediction" & ScreenName == "Screen 1"')
        pred2Corr = temp1['Correct'].sum()
        if verbose:
            print("correct predictions 2:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(pred2Corr))
#             if (pred2Corr > 0):
#                 print("Predictions2: ")
#                 print(pred2Corr)

        temp1 = temp.query('ZoneType == "response_button_text" & display == "Regions" & ScreenName == "Screen 1"')
        regTot = len(temp1)
        regCorr = temp1['Correct'].sum()
        if verbose:
            print("correct regions:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(regCorr))

        temp1 = temp.query('AgentNum == 1 & ZoneType == "response_button_text" & display == "Regions" & ScreenName == "Screen 1"')
        reg1Tot = len(temp1)
        reg1Corr = temp1['Correct'].sum()
        if verbose:
            print("correct regions 1:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(reg1Corr))
#             if (reg1Corr > 0):
#                 print("Regions1: ")
#                 print(reg1Corr)

        temp1 = temp.query('AgentNum == 2 & ZoneType == "response_button_text" & display == "Regions" & ScreenName == "Screen 1"')
        reg2Corr = temp1['Correct'].sum()
        if verbose:
            print("correct regions 2:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(reg2Corr))
#             if (reg2Corr > 0):
#                 print("Regions2: ")
#                 print(reg2Corr)

        temp1 = temp.query('ZoneType == "response_button_text" & display == "NextLvl" & ScreenName == "Screen 1"')
        lvlTot = len(temp1)
        lvlCorr = temp1['Correct'].sum()
        if verbose:
            print("correct lvl:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(lvlCorr))

        temp1 = temp.query('AgentNum == 1 & ZoneType == "response_button_text" & display == "NextLvl" & ScreenName == "Screen 1"')
        lvl1Tot = len(temp1)
        lvl1Corr = temp1['Correct'].sum()
        if verbose:
            print("correct lvl 1:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(lvl1Corr))
#             if (lvl1Corr > 0):
#                 print("Lvl1: ")
#                 print(lvl1Corr)

        temp1 = temp.query('AgentNum == 2 & ZoneType == "response_button_text" & display == "NextLvl" & ScreenName == "Screen 1"')
        lvl2Corr = temp1['Correct'].sum()
        if verbose:
            print("correct lvl 2:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(lvl2Corr))
#             if (lvl2Corr > 0):
#                 print("Lvl2: ")
#                 print(lvl2Corr)

        temp1 = temp.query('ZoneType == "response_button_text" & display == "6_buttons" & ScreenName == "Screen 1"')
        gTot = len(temp1)
        gCorr = temp1['Correct'].sum()
        if verbose:
            print("correct ghosts:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(gCorr))

        temp1 = temp.query('AgentNum == 1 & ZoneType == "response_button_text" & display == "6_buttons" & ScreenName == "Screen 1"')
        g1Tot = len(temp1)
        g1Corr = temp1['Correct'].sum()
        if verbose:
            print("correct ghosts 1:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(g1Corr))
#             if (g1Corr > 0):
#                 print("Ghosts1: ")
#                 print(g1Corr)

        temp1 = temp.query('AgentNum == 2 & ZoneType == "response_button_text" & display == "6_buttons" & ScreenName == "Screen 1"')
        g2Corr = temp1['Correct'].sum()
        if verbose:
            print("correct ghosts 2:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(g2Corr))
#             if (g2Corr > 0):
#                 print("Ghosts2: ")
#                 print(g2Corr)

        temp1 = temp.query('ZoneType == "response_button_text" & display == "CompareSectionVideoOnly" & ScreenName == "Screen 1"')
        compTot = len(temp1)
        compCorr = temp1['Correct'].sum()
        if verbose:
            print("correct comp:")
            print(temp1[['display','Response','Correct']])
            print(">>>>>>>>>>>.............Counted: " + str(compCorr))
#             if (compCorr > 8):
#                 print("Compare: ")
#                 print(compCorr)

        timeTakenMain_All = (temp.iloc[-1]['UTCTimestamp']-temp.iloc[0]['UTCTimestamp'])/1000
        temp1 = temp.query('(display == "Vis" | display == "Prediction" | display == "Regions" | \
                            display == "NextLvl" | display == "6_buttons" | display == "likerts" \
                            | display == "TextScreen" | display == "CompareSectionVideoOnly" \
                            | display == "CompareConfidenceVidOnly" | display == "CompareTextEntry") & SpreadsheetRow > 18')

        timeTakenMain_AgentsOnly = (temp1.iloc[-1]['UTCTimestamp']-temp1.iloc[0]['UTCTimestamp'])/1000
        timeTaken_Consent = (tmpConsent.iloc[-1]['UTCTimestamp']-tmpConsent.iloc[0]['UTCTimestamp'])/1000
        timeTaken_Demographics = (tmpDemo_L.iloc[-1]['UTCTimestamp']-tmpDemo_L.iloc[0]['UTCTimestamp'])/1000      
        timeTaken_AI = (tmpAI_L.iloc[-1]['UTCTimestamp']-tmpAI_L.iloc[0]['UTCTimestamp'])/1000
        time_TOTAL = (temp.iloc[-1]['UTCTimestamp']-(tmpConsent.iloc[0]['UTCTimestamp']))/1000
        
        temp1 = temp.query('AgentNum == 1')
        timeAgent1 = (temp1.iloc[-1]['UTCTimestamp']-temp1.iloc[0]['UTCTimestamp'])/1000
        temp1 = temp.query('AgentNum == 2')
        timeAgent2 = (temp1.iloc[-1]['UTCTimestamp']-temp1.iloc[0]['UTCTimestamp'])/1000
        temp1 = temp.query('AgentNum == 3')
        timeAgent3 = (temp1.iloc[-1]['UTCTimestamp']-temp1.iloc[0]['UTCTimestamp'])/1000

        vidPlayed = len(temp[temp['Response']=='VIDEO PLAYING EVENT FIRED'])
        # get average # times vid played for these visuals in this version
        vis_l = taskDF[taskDF['VisCond']==temp.VisCond.unique()[0]]
        vis_l = vis_l[vis_l['ExperimentVersion']==temp.ExperimentVersion.unique()[0]]
        vidDF = vis_l[vis_l['Response']=='VIDEO PLAYING EVENT FIRED']
        avgVidPlays = len(vidDF)/len(vidDF.ParticipantPrivateID.unique())
        
        

        part = {p_num : {'ParticipantPublicID' : temp.ParticipantPublicID.unique()[0],
                         'difficulty' : lvl, 
                         'visuals' : vc,
                         'agents' : ag, 
                         'QsTotal' : (predTot + regTot + lvlTot + gTot + compTot),
                         'QsCorrect' : (predCorr + regCorr + lvlCorr + gCorr + compCorr),
                         
                         'Agent_QsTotal' : (predTot + regTot + lvlTot + gTot),
                         'Agent_QsCorrect' : (predCorr + regCorr + lvlCorr + gCorr),
                         
                         'Agent_QsWO_GhostsTotal' : (predTot + regTot + lvlTot),
                         'Agent_QsWO_GhostsCorrect' : (predCorr + regCorr + lvlCorr),
                         
                         'Qs1_Correct' : (pred1Corr + reg1Corr + lvl1Corr + g1Corr),
                         'Qs2_Correct' : (pred2Corr + reg2Corr + lvl2Corr + g2Corr),
                         'QsPart_Total' : (pred1Tot + reg1Tot + lvl1Tot + g1Tot),

                         'predictionsCorrect' : predCorr, 
                         'predictionsTotal' : predTot,
                         'predictions1Correct' : pred1Corr, 
                         'predictions2Correct' : pred2Corr, 
                         'predictionsHalfTotal' : pred1Tot,

                         'regionsCorrect' : regCorr,
                         'regionsTotal' : regTot,
                         'regions1Correct' : reg1Corr,
                         'regions2Correct' : reg2Corr,
                         'regionsHalfTotal' : reg1Tot,

                         'levelCorrect' : lvlCorr, 
                         'levelTotal' : lvlTot,
                         'level1Correct' : lvl1Corr,
                         'level2Correct' : lvl2Corr,
                         'levelHalfTotal' : lvl1Tot,

                         'ghostsCorrect' : gCorr, 
                         'ghostsTotal' : gTot,
                         'ghosts1Correct' : g1Corr,
                         'ghosts2Correct' : g2Corr,
                         'ghostsHalfTotal' : g1Tot,

                         'comparisonsCorrect' : compCorr, 
                         'comparisonsTotal' : compTot,
                         'timeTakenMain_All': timeTakenMain_All,
                         'timeTakenMain_AgentsOnly': timeTakenMain_AgentsOnly,
                         'timeTaken_Consent': timeTaken_Consent,
                         'timeTaken_Demographics': timeTaken_Demographics,
                         'timeTaken_AI': timeTaken_AI,
                         'timeTaken_Agent1': timeAgent1,
                         'timeTaken_Agent2': timeAgent2,
                         'timeTaken_Agent3': timeAgent3,
                         'time_TOTAL': time_TOTAL,
                         'vidPlayed': vidPlayed,
                         'avgVidPlays': avgVidPlays,
                         'ratioVidPlaysToAvg': ((vidPlayed/avgVidPlays)*100),
                         'ExperimentVersion': temp.ExperimentVersion.unique()[0],
                        }} 
        # Using update() 
        # Update nested dictionary keys 
        sm_data.update(part) 
        if verbose:
            if (temp.ExperimentVersion.unique()[0] > 75):
                print(temp.ExperimentVersion.unique())
            print("....Q1 correct:")
            print((pred1Corr + reg1Corr + lvl1Corr + g1Corr))
            print("Q2 correct:")
            print((pred2Corr + reg2Corr + lvl2Corr + g2Corr))
            print("\n")
        
#Once all done, convert to DF
slimDF = pd.DataFrame.from_dict(sm_data, orient='index')
slimDF.index.name = 'ParticipantPrivateID'
# slimDF.head(5)
print(len(slimDF.ParticipantPublicID.unique()))


245


In [11]:
slimDF['totPerc'] = (slimDF['QsCorrect']/slimDF['QsTotal']) * 100
slimDF['agTotPerc'] = (slimDF['Agent_QsCorrect']/slimDF['Agent_QsTotal']) * 100
slimDF['tot1Perc'] = (slimDF['Qs1_Correct']/slimDF['QsPart_Total']) * 100
slimDF['tot2Perc'] = (slimDF['Qs2_Correct']/slimDF['QsPart_Total']) * 100

slimDF['Agent_QsWO_GhostsPerc'] = (slimDF['Agent_QsWO_GhostsCorrect']/slimDF['Agent_QsWO_GhostsTotal']) * 100

slimDF['predPerc'] = (slimDF['predictionsCorrect']/slimDF['predictionsTotal']) * 100
slimDF['pred1Perc'] = (slimDF['predictions1Correct']/slimDF['predictionsHalfTotal']) * 100
slimDF['pred2Perc'] = (slimDF['predictions2Correct']/slimDF['predictionsHalfTotal']) * 100

slimDF['regPerc'] = (slimDF['regionsCorrect']/slimDF['regionsTotal']) * 100
slimDF['reg1Perc'] = (slimDF['regions1Correct']/slimDF['regionsHalfTotal']) * 100
slimDF['reg2Perc'] = (slimDF['regions2Correct']/slimDF['regionsHalfTotal']) * 100

slimDF['lvlPerc'] = (slimDF['levelCorrect']/slimDF['levelTotal']) * 100
slimDF['lvl1Perc'] = (slimDF['level1Correct']/slimDF['levelHalfTotal']) * 100
slimDF['lvl2Perc'] = (slimDF['level2Correct']/slimDF['levelHalfTotal']) * 100

slimDF['ghostPerc'] = (slimDF['ghostsCorrect']/slimDF['ghostsTotal']) * 100
slimDF['ghost1Perc'] = (slimDF['ghosts1Correct']/slimDF['ghostsHalfTotal']) * 100
slimDF['ghost2Perc'] = (slimDF['ghosts2Correct']/slimDF['ghostsHalfTotal']) * 100

slimDF['compPerc'] = (slimDF['comparisonsCorrect']/slimDF['comparisonsTotal']) * 100

print(len(slimDF))
print(slimDF.ExperimentVersion.unique())
print(slimDF.pred1Perc.unique())

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
[ 33.33333333   0.         100.          66.66666667]


In [12]:
print(taskDF.columns)
print(len(slimDF))

Index(['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp',
       'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion',
       'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID',
       'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus',
       'ParticipantCompletionCode', 'ParticipantExternalSessionID',
       'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS',
       'ParticipantBrowser', 'ParticipantMonitorSize',
       'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion',
       'VisCond_DIFF', 'AgentCond_BlurVis_DIFF', 'AgentCond_Vis_DIFF',
       'AgentCond_Blur_DIFF', 'checkpoint_ph4w', 'checkpoint_drrq',
       'AgentCond_Norm_DIFF', 'checkpoint_6gsl', 'checkpoint_apgi', 'LVL',
       'VisCond_EASY', 'AgentCond_Norm_EASY', 'AgentCond_Vis_EASY',
       'AgentCond_BlurVis_EASY', 'AgentCond_Blur_EASY', 'Spreadsheet',
       'SpreadsheetName', 'SpreadsheetRow', 'TrialNumber', 'ScreenNumber',
       'Scre

In [13]:
# Try merging AI and Demo into Slim

# demoDF_S.set_index('ParticipantPrivateID')
# aiDF_S.set_index('ParticipantPrivateID')
print("Before:")
print(demoDF_S.columns)
print(slimDF.shape)
slimDF = pd.merge(slimDF,demoDF_S, how="inner", on='ParticipantPrivateID')
slimDF = pd.merge(slimDF,aiDF_S, how="inner", on='ParticipantPrivateID')
print("After:")
print(slimDF.columns.values)
print(slimDF.shape)
print(len(slimDF.ParticipantPublicID.unique()))
print(slimDF.ExperimentVersion.unique())

Before:
Index(['EventIndex', 'UTCTimestamp', 'UTCDate', 'LocalTimestamp',
       'LocalTimezone', 'LocalDate', 'ExperimentID', 'ExperimentVersion',
       'TreeNodeKey', 'RepeatKey', 'ScheduleID', 'ParticipantPublicID',
       'ParticipantPrivateID', 'ParticipantStartingGroup', 'ParticipantStatus',
       'ParticipantCompletionCode', 'ParticipantExternalSessionID',
       'ParticipantDeviceType', 'ParticipantDevice', 'ParticipantOS',
       'ParticipantBrowser', 'ParticipantMonitorSize',
       'ParticipantViewportSize', 'Checkpoint', 'TaskName', 'TaskVersion',
       'randomiser_8rx4', 'randomiser_ndwh', 'randomiser_fy7r',
       'randomiser_7vb5', 'checkpoint_ph4w', 'checkpoint_drrq',
       'randomiser_vfim', 'checkpoint_6gsl', 'checkpoint_apgi',
       'randomiser_k74v', 'randomiser_zcqq', 'randomiser_9z8b',
       'randomiser_xnr7', 'randomiser_l6v8', 'randomiser_dwc2',
       'Randomisequestionnaireelements?', 'age_info', 'age_info_text',
       'age_info_quantised', 'gender_id',

In [14]:
print(slimDF.ExperimentVersion.unique())
print(slimDF[slimDF['ParticipantPublicID']=='5e4ab1b4ca765d000edebd15'])
# Try merging prolific_exports_info into Slim
# load up xlsx 
infoDF = pd.read_excel('/Volumes/Britt_SSD/FromMac/Documents/School/Thesis/Gorilla/prolific_exports_info.xlsx')
# demoDF_S.set_index('ParticipantPrivateID')
# aiDF_S.set_index('ParticipantPrivateID')
print("Before:")
print(infoDF.columns)
print(slimDF.shape)
slimDF = pd.merge(slimDF,infoDF, how="inner", on='ParticipantPublicID')
# slimDF = pd.merge(slimDF,aiDF_S, how="inner", on='ParticipantPrivateID', left_index=True)
print("After:")
print(slimDF.columns.values)
print(slimDF.shape)
print(len(slimDF.ParticipantPublicID.unique()))

print(slimDF.ExperimentVersion.unique())
print(slimDF[slimDF['ParticipantPublicID']=='5e4ab1b4ca765d000edebd15'])

[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
  ParticipantPrivateID     ParticipantPublicID_x difficulty        visuals  \
3            3135747.0  5e4ab1b4ca765d000edebd15  Difficult  Blur_Plus_Vis   

    agents  QsTotal  QsCorrect  Agent_QsTotal  Agent_QsCorrect  \
3  700_300       20        8.0             16              5.0   

   Agent_QsWO_GhostsTotal  ...  ai_involvement_text  ai_involvement_other  \
3                      12  ...                 None                  None   

           ai_opinion  ai_opinion_quantised  AI_longForm  \
3  Extremely Positive                   5.0         None   

       pacman_experience  pacman_experience_text  pacman_experience_quantised  \
3  More than 5 years ago                     NaN                          3.0   

   ENDQUESTIONNAIRE_y  branch_4zei_y  
3             99806.0           None  

[1 rows x 176 columns]
Before:
Index(['ProlificName', 'session_id', 'ParticipantPublicID', 'status',
       'started_datetime', 'completed_date_time',

In [15]:
# # drop particpants who have not been accepted (rejected or timed out)
# #options: approved, awaiting review,rejected, returned, timed-out
# print(slimDF.ExperimentVersion.unique())
# print(slimDF.status.unique())
# # get names of indexes for which to drop
# pID_removed = slimDF[(slimDF['status']=='REJECTED') | 
#                       (slimDF['status']=='RETURNED')].ParticipantPublicID.unique()
# index_names = slimDF[(slimDF['status']!='AWAITING REVIEW') & (slimDF['status']!='APPROVED')].index
  
# # drop these given row
# # indexes from dataFrame
# tmpDF = slimDF.drop(index_names)
# print(tmpDF.status.unique())
# print(pID_removed)

# for pID in pID_removed:
    
#     demoDF_L.drop(demoDF_L[demoDF_L['ParticipantPublicID']==pID].index, inplace = True)
#     demoDF_S.drop(demoDF_S[demoDF_S['ParticipantPublicID']==pID].index, inplace = True)
    
#     consentDF_L.drop(consentDF_L[consentDF_L['ParticipantPublicID']==pID].index, inplace = True)
#     consentDF_S.drop(consentDF_S[consentDF_S['ParticipantPublicID']==pID].index, inplace = True)
    
#     aiDF_L.drop(aiDF_L[aiDF_L['ParticipantPublicID']==pID].index, inplace = True)
#     aiDF_S.drop(aiDF_S[aiDF_S['ParticipantPublicID']==pID].index, inplace = True)
    

In [16]:
slimDF['MediaAI'] = np.where(slimDF['ai_involvement_1']=='I know AI mostly from the media',1,0)
slimDF['HomeAI'] = np.where(slimDF['ai_involvement_2']=='I often use AI in my private life',1,0)
slimDF['AtWorkAI'] = np.where(slimDF['ai_involvement_3']=='I use AI technology at my work',1,0)
slimDF['ClassOnAI'] = np.where(slimDF['ai_involvement_4']=='I took at least one AI course',1,0)
slimDF['DevelopAI'] = np.where(slimDF['ai_involvement_5']=='I work on or research AI-related technology',1,0)
slimDF['NoneAI'] = np.where(slimDF['ai_involvement_6']=='None of the above',1,0)
slimDF['OtherAI'] = np.where(slimDF['ai_involvement_other']=='Other (please specify)',1,0)

print(len(slimDF.ParticipantPrivateID.unique()))

245


In [17]:
tmpDF = slimDF.drop(columns=[
 'EventIndex_x',
 'UTCTimestamp_x',
 'UTCDate_x',
 'LocalTimestamp_x',
 'LocalTimezone_x',
 'LocalDate_x',
 'ExperimentID_x',
 'RepeatKey_x',
 'ScheduleID_x',
 'ParticipantPublicID_y',
 'ParticipantStartingGroup_x',
 'ParticipantStatus_x',
 'ParticipantCompletionCode_x',
 'ParticipantExternalSessionID_x',
 'ParticipantDeviceType_x',
 'ParticipantDevice_x',
 'ParticipantOS_x',
 'ParticipantBrowser_x',
 'ParticipantMonitorSize_x',
 'ParticipantViewportSize_x',
 'Checkpoint_x',
 'TaskName_x',
 'TaskVersion_x',
 'randomiser_8rx4_x',
 'randomiser_ndwh_x',
 'randomiser_fy7r_x',
 'randomiser_7vb5_x',
 'checkpoint_ph4w_x',
 'checkpoint_drrq_x',
 'randomiser_vfim_x',
 'checkpoint_6gsl_x',
 'checkpoint_apgi_x',
 'randomiser_k74v_x',
 'randomiser_zcqq_x',
 'randomiser_9z8b_x',
 'randomiser_xnr7_x',
 'randomiser_l6v8_x',
 'randomiser_dwc2_x',
 'Randomisequestionnaireelements?_x',
 'ENDQUESTIONNAIRE_x',
 'branch_4zei_x',
 'EventIndex_y',
 'UTCTimestamp_y',
 'UTCDate_y',
 'LocalTimestamp_y',
 'LocalTimezone_y',
 'LocalDate_y',
 'ExperimentID_y',
 'ExperimentVersion_y',
 'TreeNodeKey_y',
 'RepeatKey_y',
 'ScheduleID_y',
 'ParticipantStartingGroup_y',
 'ParticipantStatus_y',
 'ParticipantCompletionCode_y',
 'ParticipantExternalSessionID_y',
 'ParticipantDeviceType_y',
 'ParticipantDevice_y',
 'ParticipantOS_y',
 'ParticipantBrowser_y',
 'ParticipantMonitorSize_y',
 'ParticipantViewportSize_y',
 'Checkpoint_y',
 'TaskName_y',
 'TaskVersion_y',
 'randomiser_8rx4_y',
 'randomiser_ndwh_y',
 'randomiser_fy7r_y',
 'randomiser_7vb5_y',
 'checkpoint_ph4w_y',
 'checkpoint_drrq_y',
 'randomiser_vfim_y',
 'checkpoint_6gsl_y',
 'checkpoint_apgi_y',
 'randomiser_k74v_y',
 'randomiser_zcqq_y',
 'randomiser_9z8b_y',
 'randomiser_xnr7_y',
 'randomiser_l6v8_y',
 'randomiser_dwc2_y',
 'Randomisequestionnaireelements?_y',
 'ENDQUESTIONNAIRE_y',
 'branch_4zei_y',
 'session_id',
 'started_datetime',
 'completed_date_time',
 'time_taken',
 'reviewed_at_datetime',
 'entered_code',
 'Subject',
'ParticipantPublicID_x'])

In [18]:
# 'started_datetime' 'completed_date_time' 'time_taken'
#slimDF['started_datetime'] = slimDF['started_datetime'].date()
#slimDF['completed_date_time'] = slimDF['completed_date_time'].apply(lambda x: x.value)
#slimDF['time_taken'] = slimDF['time_taken'].apply(lambda x: x.value)
tmpDF.convert_dtypes()
tmpDF.dtypes.to_dict()

{'ParticipantPrivateID': dtype('O'),
 'difficulty': dtype('O'),
 'visuals': dtype('O'),
 'agents': dtype('O'),
 'QsTotal': dtype('int64'),
 'QsCorrect': dtype('float64'),
 'Agent_QsTotal': dtype('int64'),
 'Agent_QsCorrect': dtype('float64'),
 'Agent_QsWO_GhostsTotal': dtype('int64'),
 'Agent_QsWO_GhostsCorrect': dtype('float64'),
 'Qs1_Correct': dtype('float64'),
 'Qs2_Correct': dtype('float64'),
 'QsPart_Total': dtype('int64'),
 'predictionsCorrect': dtype('float64'),
 'predictionsTotal': dtype('int64'),
 'predictions1Correct': dtype('float64'),
 'predictions2Correct': dtype('float64'),
 'predictionsHalfTotal': dtype('int64'),
 'regionsCorrect': dtype('float64'),
 'regionsTotal': dtype('int64'),
 'regions1Correct': dtype('float64'),
 'regions2Correct': dtype('float64'),
 'regionsHalfTotal': dtype('int64'),
 'levelCorrect': dtype('float64'),
 'levelTotal': dtype('int64'),
 'level1Correct': dtype('float64'),
 'level2Correct': dtype('float64'),
 'levelHalfTotal': dtype('int64'),
 'ghost

In [19]:
# # Can we get mean time spent on each row of the spreadsheet for each visual?
# rows_Dict = OrderedDict()

# for v_index, vis in enumerate(taskDF.VisCond.unique()):
#     temp1 = taskDF[taskDF['VisCond']==vis]
#     for r_index, row in enumerate(temp1.SpreadsheetRow.unique()):
#         temp2 = temp1[temp1['SpreadsheetRow']==row]
#         ls_tmp = []
        
#         # for each row, fill with info on each participant
#         for p_index, pID in enumerate(temp2.ParticipantPrivateID.unique()):
#             temp3 = temp2[temp2['ParticipantPrivateID']==pID]
#             temp3.set_index('EventIndex')
#             timeOnRowInSeconds = (temp3.iloc[-1]['UTCTimestamp']-temp3.iloc[0]['UTCTimestamp'])/1000
# #             print("Participant " + str(pID) + " spent " + str(timeOnRow) + " seconds on row " + str(row))
#             ls_tmp.append(timeOnRowInSeconds)
        
#         if (len(ls_tmp) > 0):
#             # now add mean, min, max, and std to ordered dictionary for the row 
#     #         new_result = {'new_key1':  {'key': 'value',..}
#             row_stats = { int(row): {
#                 'VisCond': vis, 
#                 'mean': statistics.mean(ls_tmp),
#                 'min': min(ls_tmp),
#                 'max': max(ls_tmp),
#                 'std': statistics.stdev(ls_tmp)
#             }}
#             rows_Dict.update(row_stats)
# #             print("Mean for vis " + str(vis) + " of row " + str(row) + " is " + str(statistics.mean(ls_tmp)))
        
# rowsDF = pd.DataFrame.from_dict(rows_Dict, orient='index')


In [20]:
tempDir = '/Volumes/Britt_SSD/ReDoEvals3/'
tempTopDir = os.path.join(tempDir, 'UserStudy')
if not os.path.isdir(tempTopDir):
    os.mkdir(tempTopDir)
tempTopDir = os.path.join(tempDir, 'UserStudy')

# Write Short Task DF to parquet:
filePath = os.path.join(tempTopDir, 'GP_Main_Short.parquet')
tmpDF.to_parquet(path=filePath,compression='brotli')
# table = pa.Table.from_pandas(tmpDF, safe=False)
# Parquet with Brotli compression
# pq.write_table(table, filePath, compression='BROTLI')
filePath = os.path.join(tempTopDir, 'GP_Main_Short.csv')
tmpDF.to_csv(filePath)

# Write ROW INFO DF to parquet:
# filePath = os.path.join(tempTopDir, 'GP_RowsInfo.parquet')
# # mainDF.to_parquet(path=filePath,compression='brotli')
# table = pa.Table.from_pandas(rowsDF, safe=False)
# # Parquet with Brotli compression
# pq.write_table(table, filePath, compression='BROTLI')

In [21]:
taskDF.iloc[:,40:70].tail(20)

,AgentCond_Blur_EASY,Spreadsheet,SpreadsheetName,SpreadsheetRow,TrialNumber,ScreenNumber,ScreenName,ZoneName,ZoneType,ReactionTime,...,randomise_blocks,randomise_trials,display,ANSWER,maxVidRepeats,B1,B2,B3,B4,I1
75795,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,63.0,7,1.0,Screen 1,ContinueButton,continue_button,86909.190,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75796,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid2,content_video,205.895,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75797,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid2,content_video,234.205,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75798,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid1,content_video,235.590,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75799,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid1,content_video,263.620,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75800,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid2,content_video,445.630,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75801,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid2,content_video,190.355,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75802,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid1,content_video,450.125,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75803,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,Vid1,content_video,202.556,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None
75804,None,STD_300_VS_PP_700_Blur,STD_300_VS_PP_700_Blur,64.0,8,1.0,Screen 1,TextBox,response_text_area,3351.945,...,NaN,NaN,CompareTextEntry,None,200.0,Agent Thompson,Agent Jackson,Either One,Neither,None


In [22]:
# taskDF.query('ZoneType == "response_text_area" & display == "CompareTextEntry"')['Response']
taskDF.query('ZoneType == "response_text_area"')['Response']

103      Although none of the graphs showed that the ia...
105                           Training progress statistics
109      It was the only AI that shown an approach to t...
111      Videos of agent thompson's behavior when ghost...
124      In their statistics, the average of their step...
                               ...                        
75756    Jackson did a better job at avoiding the ghost...
75766                                                 none
75794          Thompson used the pills better than jackson
75804                                                 none
75812                                                 None
Name: Response, Length: 2191, dtype: object

In [23]:
print(len(tmpDF.ParticipantPrivateID.unique()))

245
